In [2]:
!pip install trax 
import trax
from trax import layers as tl
from trax.fastmath import numpy as jnp

import sys
import os
import numpy as np
import textwrap

wrapper = textwrap.TextWrapper(width=70)
np.set_printoptions(threshold=sys.maxsize)

     |████████████████████████████████| 522kB 17.2MB/s 
     |████████████████████████████████| 3.4MB 41.5MB/s 
     |████████████████████████████████| 235kB 42.3MB/s 
     |████████████████████████████████| 1.2MB 41.6MB/s 
     |████████████████████████████████| 2.1MB 28.8MB/s 
     |████████████████████████████████| 3.9MB 38.8MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 368kB 38.5MB/s 
     |████████████████████████████████| 870kB 39.6MB/s 
     |████████████████████████████████| 3.3MB 39.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=ed63ecadd1b3be79ea9325b88627ed64b8cb13e0cba128d96c3fc7eda4dbb818
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


# Dataset

In [3]:
train_stream_fn = trax.data.TFDS('scientific_papers',
                                 data_dir='content/',
                                 keys=('abstract', 'article'),
                                 train=True)

eval_stream_fn = trax.data.TFDS('scientific_papers',
                                data_dir='content/',
                                keys=('abstract', 'article'),
                                train=False)

Dataset scientific_papers downloaded and prepared to content/scientific_papers/arxiv/1.1.1. Subsequent calls will reuse this data.


# Tokenization

In [4]:
def tokenize(input_str, EOS=1):
    # trax.data.tokenize takes streams and returns streams
    inputs =  next(trax.data.tokenize(iter([input_str]),
                                      vocab_dir='vocab/',
                                      vocab_file='vocab.subwords'))
    
    return list(inputs) + [EOS]


def detokenize(integers):
    s = trax.data.detokenize(integers,
                             vocab_dir='vocab/',
                             vocab_file='vocab.subwords')
    
    return wrapper.fill(s)


SEP = 0 
EOS = 1

# Tokenized inputs + targets; 0 is sep
def data_preprocessing(stream):
    for (abstract, article) in stream:
        joint = np.array(list(article) + [EOS, SEP] + list(abstract) + [EOS])
        mask = [0] * (len(list(article)) + 2) + [1] * (len(list(abstract)) + 1) # Accounting for EOS and SEP
        yield joint, joint, np.array(mask)

input_pipeline = trax.data.Serial(
    trax.data.Tokenize(vocab_dir='vocab/',
                       vocab_file='vocab.subwords'),
    data_preprocessing,
    trax.data.FilterByLength(2048)
)

# Apply
train_stream = input_pipeline(train_stream_fn())
eval_stream = input_pipeline(eval_stream_fn())

train_input, train_target, train_mask = next(train_stream)

assert sum((train_input - train_target)**2) == 0  # They are the same in LM

print(f'Mask sample:\n {train_mask} \n\n')
print('[Example][<EOS>][<pad>][Example Summary][<EOS>] \n')
print(f'Data sample:\n {detokenize(train_input)}')

Mask sample:
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 

# Bucketing

![alt text](https://sun9-62.userapi.com/impg/fuoqdikR_mdjXzKxawFUQl7mNWMiA2SHLsmvsA/UWaTaN0In-E.jpg?size=1444x976&quality=96&sign=775b3928fe8deb0c25f94926f5903efa&type=album)

In [15]:
# Batches of 16 sentences of length < 128 , 
# batches of 8 sentences of length < 256,
# batches of 4 sentences of length < 512, 
# batches of 2 sentences of length < 1024,
# batches of 1 sentence of length < 2048. 
boundaries =  [128, 256,  512, 1024]
batch_sizes = [16,    8,    4,    2, 1]


train_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes)(train_stream)

eval_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes)(eval_stream)

# Random sampling
input_batch, _, mask_batch = next(train_batch_stream)

print(input_batch.shape, '\n')
print('[article word values, ... article word values, 1 <EOS>, 0 <pad>, \
\n abstract word values, ... abstract word values, 1 <EOS>, \n 0s for padding \
(if there\'re no 0s in the end, we reached the maximum length)] \n')
print(input_batch[0])
print('\n Article:\n', detokenize(input_batch[0]))

(1, 1711) 

[article word values, ... article word values, 1 <EOS>, 0 <pad>, 
 abstract word values, ... abstract word values, 1 <EOS>, 
 0s for padding (if there're no 0s in the end, we reached the maximum length)] 

[  697    70   230 25719  5409  3601 19622     5   402   106   412   213
 23533 25901  2902  5147   697 24224     4  2652 26054     4 23662  3387
   669  4884 14272 14272 14272 27439  6774  7583  4884   669  6435  4884
 14272 14272 14272 27439  6774  7583  4884  2652  5243 13929 10018  1045
   402 12846  5065   556 23662  2627     2    39  1151  4712   320 25719
  5409  8033   132   213 10432 24255   364   598 27439  6774  7583     3
   213 17104 16071   524   527  1896   763 15986     4  2652 15661    21
  3610 17015  1791    39  1151   213  4469  1175   132   824  4590   598
   379  1248    87   286   446   783   742   320  1151  4237  8051  1248
 24529   497   121    10    32  1564  2703   364 27439  6774  7583     2
    97 17015  1791    39  1771    28  9819    70  10

# Transformer LM

$$
\text { Attention }(Q, K, V)=\operatorname{softmax}\left(\frac{Q K^{T}}{\sqrt{d_{k}}}+{M}\right) V\tag{1}\
$$

In [28]:
loop = training_loop(TransformerLM, train_batch_stream, eval_batch_stream)
loop.run(10)


Step      1: Total number of trainable weights: 316336
Step      1: Ran 1 train steps in 10.74 secs
Step      1: train CrossEntropyLoss |  10.41117954
Step      1: eval  CrossEntropyLoss |  10.41211128
Step      1: eval          Accuracy |  0.00000000

Step     10: Ran 9 train steps in 99.00 secs
Step     10: train CrossEntropyLoss |  10.41329575
Step     10: eval  CrossEntropyLoss |  10.41292191
Step     10: eval          Accuracy |  0.00000000


In [30]:
model = TransformerLM(mode='eval')